In [0]:
# /FileStore/tables/beauty4.csv
import os.path
# baseDir = os.path.join('data')
# inputPath = os.path.join('SENG501', 'lab1', 'deerfoot.csv')
fileName = '/FileStore/tables/beauty4.csv'

reviewsRDD = sc.textFile(fileName)
trailed = reviewsRDD.map(lambda x: x.rstrip("'"))
print(trailed.take(1))
print(trailed.count())

["'5','True','09 1, 2016','A3CIUOJXQ5VDQ2','B0000530HU','{''Size:'': '' 7.0 oz'', ''Flavor:'': '' Classic Ice Blue''}','Shelly F','As advertised. Reasonably priced','Five Stars','1472688000','',"]
5269


In [0]:
# TODO: Replace <FILL IN> with appropriate code
def extractFields(reviewsRDD):
    fieldsList = reviewsRDD.split("','")
    print(len(fieldsList))
    rate = fieldsList[0].replace("'","")
    return (rate, fieldsList[7], fieldsList[8])
     
print(extractFields(reviewsRDD.take(1)[0]))

12
('5', 'As advertised. Reasonably priced', 'Five Stars')


In [0]:
reviewsRatingRDD = reviewsRDD.map(extractFields)
print(reviewsRatingRDD.take(3))

[('5', 'As advertised. Reasonably priced', 'Five Stars'), ('5', 'Like the oder and the feel when I put it on my face.  I have tried other brands but the reviews from people I know they prefer the oder of this brand. Not hard on the face when dry.  Does not leave dry skin.', 'Good for the face'), ('1', 'I bought this to smell nice after I shave.  When I put it on I smelled awful.  I am 19 and I smelled like a grandmother with too much perfume.', 'Smells awful')]


### Text Preprocessing and TFIDF
###### Make all text from review-text and summary-text lowercase and remove all punctuation
###### do tfidf
###### ref: <https://spark.apache.org/docs/latest/mllib-feature-extraction.html>

In [0]:
import string
from pyspark.mllib.feature import HashingTF, IDF

# remove punctuation and make everything lowercase

def prepData(rdd):
    r1 = rdd[1].translate(str.maketrans('','',string.punctuation))
    r2 = rdd[2].translate(str.maketrans('','',string.punctuation))
    rev = r2.lower() + ' ' + r1.lower()
    # rev = rev.translate(str.maketrans('','',string.punctuation))
    label = int(rdd[0])
    return (label, rev)


reviewsData = reviewsRatingRDD.map(prepData)
print(reviewsData.take(2))



[(5, 'five stars as advertised reasonably priced'), (5, 'good for the face like the oder and the feel when i put it on my face  i have tried other brands but the reviews from people i know they prefer the oder of this brand not hard on the face when dry  does not leave dry skin')]


In [0]:
rWordList2 = reviewsData.map(lambda x: x[1].split())
# print(rWordList2.take(2))

tf = HashingTF()
tfRDD = tf.transform(rWordList2)
tfRDD.cache()
idf = IDF().fit(tfRDD)
tfidf = idf.transform(tfRDD)


In [0]:
print(reviewsData.take(3)[1][1])
print(rWordList2.take(3)[1])

like the oder and the feel when i put it on my face  i have tried other brands but the reviews from people i know they prefer the oder of this brand not hard on the face when dry  does not leave dry skin good for the face
['like', 'the', 'oder', 'and', 'the', 'feel', 'when', 'i', 'put', 'it', 'on', 'my', 'face', 'i', 'have', 'tried', 'other', 'brands', 'but', 'the', 'reviews', 'from', 'people', 'i', 'know', 'they', 'prefer', 'the', 'oder', 'of', 'this', 'brand', 'not', 'hard', 'on', 'the', 'face', 'when', 'dry', 'does', 'not', 'leave', 'dry', 'skin', 'good', 'for', 'the', 'face']


In [0]:
# testing
r1 = rWordList2.zip(tfidf)
s = r1.take(3)
print(s[0][1])
print(s[0][0])
# print(r1.take(1)[0])
# print(tfidf.take(2))

(1048576,[13727,246493,665090,680533,761597,929834],[1.2679632993974759,6.778026172307353,5.168588259873252,5.679413883639243,1.1716115485649428,1.8952242497209821])
['as', 'advertised', 'reasonably', 'priced', 'five', 'stars']


In [0]:
r2 = reviewsData.zip(tfidf)
m = r2.take(3)
print(m[0][0][0])
print(m[0][1])

5
(1048576,[13727,246493,665090,680533,761597,929834],[1.2679632993974759,6.778026172307353,5.168588259873252,5.679413883639243,1.1716115485649428,1.8952242497209821])


In [0]:
"""
rText = reviewsRatingRDD.map(prepText)
print(rText.take(2))

rWordList = rText.map(lambda x: x.split(" "))
tf = HashingTF().transform(rWordList)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)
"""

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
# idfIgnore = IDF(minDocFreq=2).fit(tf)
# tfidfIgnore = idfIgnore.transform(tf)

# Doing ML

## Random Forest Model
##### ref: <https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.mllib.tree.RandomForest.html#pyspark.mllib.tree.RandomForest>

In [0]:
from pyspark.mllib.regression import LabeledPoint


def parsePoint(line):
    val = LabeledPoint(line[0][0], line[1])
    return val

featurizedRDD = r2.map(parsePoint)

In [0]:
print(featurizedRDD.take(1))

[LabeledPoint(5.0, (1048576,[13727,246493,665090,680533,761597,929834],[1.2679632993974759,6.778026172307353,5.168588259873252,5.679413883639243,1.1716115485649428,1.8952242497209821]))]


In [0]:
sample = r2.map(parsePoint)
print(sample.take(1)[0].label)
print(sample.take(1)[0].features)
print(len(sample.take(1)[0].features))
print(type(sample.take(1)[0].features))

<class 'pyspark.mllib.linalg.SparseVector'>


#### Splitting Data

In [0]:
weights = [0.6, 0.2, 0.2]
seed = 42
trainData, valData, testData = featurizedRDD.randomSplit(weights, seed)

trainData.cache()
valData.cache()
testData.cache()

nTrain = trainData.count()
nVal = valData.count()
nTest = testData.count()

print(nTrain, nVal, nTest, nTrain + nVal + nTest)

3151 1042 1076 5269


### Start training model
##### ref <https://spark.apache.org/docs/latest/mllib-ensembles.html#classification>

In [0]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
import numpy as np

model = RandomForest.trainClassifier(trainData, 6, {}, 10)


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3857930735326542> in <module>
      8 labelsAndPred = valData.map(lambda lp: lp.label).zip(predictions)
      9 
---> 10 Ev = labelsandPred.filter(lambda lp: lp[0] != lp[1].count() / float (valData.count()))
     11 print('Test Err:' + str(Ev))
     12 print('Forest Model:')

NameError: name 'labelsandPred' is not defined

In [0]:
model2 = RandomForest.trainClassifier(trainData, numClasses=6, categoricalFeaturesInfo={}, numTrees=20, maxDepth=4)

In [0]:
model3 = RandomForest.trainClassifier(trainData, numClasses=6, categoricalFeaturesInfo={}, numTrees=10, maxDepth=7)

In [0]:
#evaluation
predictions = model.predict(valData.map(lambda x: x.features))
labelsAndPred = valData.map(lambda lp: lp.label).zip(predictions)

predictions2 = model2.predict(valData.map(lambda x: x.features))
labelsAndPred2 = valData.map(lambda lp: lp.label).zip(predictions2)

predictions3 = model3.predict(valData.map(lambda x: x.features))
labelsAndPred3 = valData.map(lambda lp: lp.label).zip(predictions3)


In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(labelsAndPred)
acc = metrics.accuracy * 100
err = 100 - acc
print("Model 1: ")
print("Accuracy: %.2f%%" % acc)
print("Error: %.2f%%" % err)

metrics2 = MulticlassMetrics(labelsAndPred2)
acc2 = metrics2.accuracy * 100
err2 = 100 - acc2

print("Model 2: ")
print("Accuracy: %.2f%%" % acc2)
print("Error: %.2f%%" % err2)

metrics3 = MulticlassMetrics(labelsAndPred3)
acc3 = metrics3.accuracy * 100
err3 = 100 - acc3

print("Model 3: ")
print("Accuracy: %.2f%%" % acc3)
print("Error: %.2f%%" % err3)

Model 1: 
Accuracy: 88.29%
Error: 11.71%
Model 2: 
Accuracy: 88.29%
Error: 11.71%
Model 3: 
Accuracy: 88.29%
Error: 11.71%


In [0]:

predictionAndLabels = valData.map(lambda lp: (float(model.predict(lp.features)), lp.label))
metrics2 = MulticlassMetrics(predictionAndLabels)
prec = metrics2.precision()
print("precision: %.2f%%" % prec * 100)

Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/serializers.py", line 473, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
  File "/databricks/spark/python/pyspark/context.py", line 369, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.


---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
/databricks/spark/python/pyspark/serializers.py in dumps(self, obj)
    472         try:
--> 473             return cloudpickle.dumps(obj, pickle_protocol)
    474         except pickle.PickleError:

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dumps(obj, protocol, buffer_callback)
     72             )
---> 73             cp.dump(obj)
     74             return file.getvalue()

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dump(self, obj)
    562         try:
--> 563             return Pickler.dump(self, obj)
    564         except RuntimeError as e:

/databricks/spark/python/pyspark/context.py in __getnewargs__(self)
    368         # This method is called when attempting to pickle SparkContext, which is always an error:
--> 369         raise RuntimeError(
    370             "It appears th